In [1]:
# install packages
!pip install pandas
!pip install gurobipy

In [2]:
# import libraries
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## Import Data


In [3]:
url = "/content/drive/MyDrive/Colab Notebooks/361IOR/P361IOR.xlsx"

order_df = pd.read_excel(url, sheet_name='orders')
order_df.head()


,product,plant,month,demand
0,1,1,February,3600
1,1,1,March,6300
2,1,2,February,4900
3,1,2,March,4200
4,2,1,February,4500


In [4]:
prodcost_df = pd.read_excel(url, sheet_name='production_cost')
prodcost_df.head()

,product,plant,process,production_cost
0,1,1,1,62
1,1,1,2,59
2,1,2,1,61
3,1,2,2,65
4,2,1,1,78


In [5]:
prodrate_df = pd.read_excel(url, sheet_name='production_rate')
prodrate_df.head()

,product,plant,process,production_rate
0,1,1,1,100
1,1,1,2,140
2,1,2,1,130
3,1,2,2,110
4,2,1,1,120


In [6]:
salesvalue_df = pd.read_excel(url, sheet_name='net_sales_value')
salesvalue_df.head()

,product,sales_value
0,1,83
1,2,112


In [7]:
extrashippingcost_df = pd.read_excel(url, sheet_name='extra_shipping_cost')
extrashippingcost_df.head()

,product,extra_shipping_cost
0,1,9
1,2,7


In [8]:
holdingcost_df = pd.read_excel(url, sheet_name='holding_cost')
holdingcost_df.head()

,product,holding_cost
0,1,3
1,2,2


In [9]:
proddays_df = pd.read_excel(url, sheet_name='production_days')
proddays_df.head()

,month,production_days
0,February,20
1,March,23


## Data preparing


In [10]:
# sets
products = sorted(order_df['product'].unique().tolist())
plants = sorted(order_df['plant'].unique().tolist())
processes = sorted(prodcost_df['process'].unique().tolist())
months = sorted(order_df['month'].unique().tolist())

In [11]:
# parameters - objective function

# sales value
revenue = {
    row.product:row.sales_value
    for row in salesvalue_df.itertuples()
}

# production cost
prodcost = {
    (row.product, row.plant, row.process):row.production_cost
    for row in prodcost_df.itertuples()
}

# holding cost
holdingcost = {
    row.product:row.holding_cost
    for row in holdingcost_df.itertuples()
}

# extra shipping cost
eshipcost = {
    row.product:row.extra_shipping_cost
    for row in extrashippingcost_df.itertuples()
}

In [12]:
# parameters - constraints

# demand
demand = {
    (row.product, row.plant, row.month):row.demand
    for row in order_df.itertuples()
}

# production rate
prodrate = {
    (row.product, row.plant, row.process):row.production_rate
    for row in prodrate_df.itertuples()
}

proddays = {
    row.month:row.production_days
    for row in proddays_df.itertuples()
}

## Formulatind and solving the model


In [13]:
# initializing model
model = gp.Model('361IOR')

# decision variables
# number of units of product i produced at plant j using process k in month m
X = model.addVars(
    products, plants, processes, months,
    vtype=GRB.INTEGER, lb=0,
    name="Produce"
)

# number of units of product i held in inventory at plant k at the end of month m
I = model.addVars(
    products, plants, months,
    vtype=GRB.INTEGER, lb=0,
    name='Inventory'
)

# number of units of product i shipped from plant j to plant l (different from j) in month m
jl = [(j, l) for j in plants for l in plants if l != j]
E = model.addVars(
    products,
    jl,
    months,
    vtype=GRB.INTEGER, lb=0,
    name="Extra_Shipping")

# number of units of product i sold by plant j in month m
S = model.addVars(
    products, plants, months,
    vtype=GRB.INTEGER, lb=0,
    name='Sales'
)

Restricted license - for non-production use only - expires 2027-11-29


In [14]:
# objective function
# net sales revenue
net_sales_revenue = gp.quicksum(revenue[i]*S[i, j, m]
                                for i in products
                                for j in plants
                                for m in months)

# production cost
production_cost = gp.quicksum(prodcost[i, j, k]*X[i, j , k, m]
                              for i in products
                              for j in plants
                              for k in processes
                              for m in months)

# inventory cost
inventory_cost = gp.quicksum(holdingcost[i]*I[i, j, m]
                             for i in products
                             for j in plants
                             for m in months)

# extra shipping cost
extra_shipping_cost = gp.quicksum(eshipcost[i]*E[i, j, l, m]
                                  for i in products
                                  for j in plants
                                  for l in plants if j != l
                                  for m in months)

# total profit
profit = net_sales_revenue - (production_cost + inventory_cost + extra_shipping_cost)

# set objective to maximize profit
model.setObjective(profit, gp.GRB.MAXIMIZE)

In [15]:
# constraints
# production rate
for i in products:
  for j in plants:
    for k in processes:
      for m in months:
        model.addConstr(
            X[i, j, k, m] <= prodrate[i, j, k]*proddays[m]
          )

# inventory balancing
prev_month = {
    months[t]: months[t-1]
    for t in range(1, len(months))
}

for i in products:
    for j in plants:
        for m_idx, m in enumerate(months):
            if m_idx == 0:
                I_prev = 0
            else:
                I_prev = I[i, j, prev_month[m]]

            model.addConstr(
                I[i, j, m]
                == I_prev
                + gp.quicksum(X[i, j, k, m] for k in processes)
                + gp.quicksum(E[i, l, j, m] for l in plants if l != j) # number of units of product i shipped from plant l to j
                - gp.quicksum(E[i, j, l, m] for l in plants if l != j)
                - S[i, j, m]
            )

# inventory capacity
for i in products:
  for m in months:
    model.addConstr(gp.quicksum(I[i, j, m] for j in plants)
                      <= 1000
                      )
# demand constraints
for i in products:
    for j in plants:
        for m in months:
            model.addConstr(S[i, j, m] <= demand[i, j, m])

In [16]:
model.optimize()

Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 36 rows, 40 columns and 84 nonzeros (Max)
Model fingerprint: 0x16f99802
Model has 40 linear objective coefficients
Variable types: 0 continuous, 40 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 6e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 30 rows and 20 columns
Presolve time: 0.00s
Presolved: 6 rows, 20 columns, 30 nonzeros
Found heuristic solution: objective 409180.00000
Variable types: 0 continuous, 20 integer (0 binary)

Root relaxation: objective 9.846700e+05, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

In [17]:
if model.Status == GRB.OPTIMAL:
    print(f"\nMaximize Profit: {model.ObjVal}")

    print(f"\nTotal Net Sales Value: {net_sales_revenue.getValue()}")
    print(f"\nTotal Production Cost: {production_cost.getValue()}")
    print(f"\nTotal Extra Shipping Cost: {extra_shipping_cost.getValue()}")
    print(f"\nTotal Inventory Cost: {inventory_cost.getValue()}")

    print("\nAll Decision Variables:")
    for v in model.getVars():
        print(f"{v.VarName}: {v.X}")
else:
    print("Optimization was not successful. Check model status.")


Maximize Profit: 984670.0

Total Net Sales Value: 3929000.0

Total Production Cost: 2941090.0

Total Extra Shipping Cost: 900.0

Total Inventory Cost: 2340.0

All Decision Variables:
Produce[1,1,1,February]: 1680.0
Produce[1,1,1,March]: 2300.0
Produce[1,1,2,February]: 2800.0
Produce[1,1,2,March]: 3220.0
Produce[1,2,1,February]: 2600.0
Produce[1,2,1,March]: 2990.0
Produce[1,2,2,February]: 2200.0
Produce[1,2,2,March]: 1210.0
Produce[2,1,1,February]: 2400.0
Produce[2,1,1,March]: 2760.0
Produce[2,1,2,February]: 2100.0
Produce[2,1,2,March]: 2640.0
Produce[2,2,1,February]: 2500.0
Produce[2,2,1,March]: 3010.0
Produce[2,2,2,February]: 2600.0
Produce[2,2,2,March]: 2990.0
Inventory[1,1,February]: 780.0
Inventory[1,1,March]: 0.0
Inventory[1,2,February]: 0.0
Inventory[1,2,March]: 0.0
Inventory[2,1,February]: -0.0
Inventory[2,1,March]: -0.0
Inventory[2,2,February]: -0.0
Inventory[2,2,March]: -0.0
Extra_Shipping[1,1,2,February]: 100.0
Extra_Shipping[1,1,2,March]: 0.0
Extra_Shipping[1,2,1,February]: